## Setup

In [ ]:
# Load required packages
library(tidyverse)

In [ ]:
# Load Fst data
load_fst_data <- function(path){
    name <- basename(path)
    win_size <- str_extract(name, "(?<=win)\\d+(?=\\.txt)")
    df <- read_delim(path, delim = "\t") %>% 
        mutate(win_size = win_size)
    return(df)
}
all_fst_df <- snakemake@input[["all_fsts"]] %>% 
    purrr::map_dfr(load_fst_data) %>% 
    mutate(win_size = as.character(win_size))

## nTrees / nSites

In [ ]:
nSites_nTrees_df <- read_csv(snakemake@input[["nSites_nTrees"]]) %>% 
    group_by(chrom, regionID) %>% 
    summarise(nsites = mean(num_sites),
              ntrees = mean(num_trees)) %>% 
    mutate(ratio = ntrees / nsites) 

nSites_nTrees_hist <- nSites_nTrees_df %>% 
    ggplot(aes(x = ratio)) +
        geom_histogram(bins = 30, color = 'black', fill = 'red') +
        ylab("Number of regions") + xlab("Ratio of # trees to # sites") +
        theme_classic() +
        theme(axis.title = element_text(size = 15),
              axis.text = element_text(size = 13))
nSites_nTrees_hist

ggsave(filename = snakemake@output[["nTrees_nSites_hist"]], plot = nSites_nTrees_hist,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

## Per-site and windowed Fst histograms

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 6)
labels <- c(`1` = "Per-site", `10000` = '10 kb windows')
site_fst_winSize_hist <- all_fst_df %>% 
    ggplot(aes(x = arg_site_fst)) +
        geom_histogram(bins = 100, color = "black", fill = "red") +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        ylab("Number of sites") + xlab("Site-based Fst from tskit (i.e. ARGs)") +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
site_fst_winSize_hist

ggsave(filename = snakemake@output[["site_fst_winSize_hist"]], plot = site_fst_winSize_hist,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

In [ ]:
all_fst_df_filtered <- all_fst_df %>% 
    filter(!(arg_site_fst < -1 & win_size == 1)) %>% 
    filter(!(no_snps < 20 & win_size == 10000))
all_fst_df_filtered

In [ ]:
site_fst_winSize_hist_filt <- all_fst_df_filtered %>% 
    ggplot(aes(x = arg_site_fst)) +
        geom_histogram(bins = 100, color = "black", fill = "red") +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        ylab("Number of sites") + xlab("Site-based Fst from tskit (i.e. ARGs)") +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
site_fst_winSize_hist_filt

ggsave(filename = snakemake@output[["site_fst_winSize_hist_filt"]], plot = site_fst_winSize_hist_filt,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

## Per-site and windowed Fst biplots

In [ ]:
branch_fst_winSize_hist_filt <- all_fst_df_filtered %>% 
    ggplot(aes(x = arg_branch_fst)) +
        geom_histogram(bins = 100, color = "black", fill = "red") +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        ylab("Number of sites") + xlab("Branch-based Fst from tskit (i.e. ARGs)") +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
branch_fst_winSize_hist_filt

ggsave(filename = snakemake@output[["branch_fst_winSize_hist_filt"]], plot = branch_fst_winSize_hist_filt,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 6)
site_gt_fst_cor_by_winSize <- all_fst_df_filtered %>% 
    ggplot(aes(x = arg_site_fst, y = gt_hudson_fst)) +
        geom_point(size = 2, alpha = 0.2) +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        geom_smooth(method = "lm", linewidth = 1, color = 'blue') +
        ylab("Hudson's Fst from Pixy (i.e. VCFs)") + xlab("Site-based Fst from tskit (i.e. ARGs)") +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
site_gt_fst_cor_by_winSize

ggsave(filename = snakemake@output[["site_gt_fst_cor_by_winSize"]], plot = site_gt_fst_cor_by_winSize,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

In [ ]:
branch_gt_fst_cor_by_winSize <- all_fst_df_filtered %>% 
    ggplot(aes(x = arg_branch_fst, y = gt_hudson_fst)) +
        geom_point(size = 2, alpha = 0.2) +
        geom_smooth(method = "lm", linewidth = 1, color = 'blue') +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        ylab("Hudson's Fst from Pixy (i.e. VCFs)") + xlab("Branch-based Fst from tskit (i.e. ARGs)") +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
branch_gt_fst_cor_by_winSize

ggsave(filename = snakemake@output[["branch_gt_fst_cor_by_winSize"]], plot = branch_gt_fst_cor_by_winSize,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

In [ ]:
site_sfs_fst_cor_by_winSize <- all_fst_df_filtered %>% 
    ggplot(aes(x = arg_site_fst, y = sfs_hudson_fst)) +
        geom_point(size = 2, alpha = 0.2) +
        geom_smooth(method = "lm", linewidth = 1, color = 'blue') +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        ylab("Hudson's Fst from ANGSD (i.e. VCFs)") + xlab("Site-based Fst from tskit (i.e. ARGs)") +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
site_sfs_fst_cor_by_winSize

ggsave(filename = snakemake@output[["site_sfs_fst_cor_by_winSize"]], plot = site_sfs_fst_cor_by_winSize,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

In [ ]:
branch_sfs_fst_cor_by_winSize <- all_fst_df_filtered %>% 
    ggplot(aes(x = arg_branch_fst, y = sfs_hudson_fst)) +
        geom_point(size = 2, alpha = 0.2) +
        geom_smooth(method = "lm", linewidth = 1, color = 'blue') +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        ylab("Hudson's Fst from ANGSD (i.e. VCFs)") + xlab("Branch-based Fst from tskit (i.e. ARGs)") +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
branch_sfs_fst_cor_by_winSize

ggsave(filename = snakemake@output[["branch_sfs_fst_cor_by_winSize"]], plot = branch_sfs_fst_cor_by_winSize,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

## Fst correlation histograms

In [ ]:
# Create dataframe with Fst correlation coefficients
all_fst_cors <- all_fst_df_filtered %>% 
    group_by(win_size, regionID) %>% 
    summarize(branch_gt_cor = cor(arg_branch_fst, gt_hudson_fst, use = "complete.obs"),
              site_gt_cor = cor(arg_site_fst, gt_hudson_fst, use = "complete.obs"),
              branch_sfs_cor = cor(arg_branch_fst, sfs_hudson_fst, use = "complete.obs"),
              site_sfs_cor = cor(arg_site_fst, sfs_hudson_fst, use = "complete.obs"))
all_fst_cors

In [ ]:
branch_gt_cor_hist_by_winSize <- all_fst_cors %>% 
    ggplot(aes(x = branch_gt_cor)) +
        geom_histogram(bins = 30, color = "black", fill = "red") +
        ylab("Number of 1Mb regions") + xlab("ARG branch Fst vs. Pixy GT Fst correlation") +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
branch_gt_cor_hist_by_winSize

ggsave(filename = snakemake@output[["branch_gt_cor_hist_by_winSize"]], plot = branch_gt_cor_hist_by_winSize,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

In [ ]:
branch_gt_cor_hist_by_winSize <- all_fst_cors %>% 
    ggplot(aes(x = branch_gt_cor)) +
        geom_histogram(bins = 30, color = "black", fill = "red") +
        ylab("Number of 1Mb regions") + xlab("ARG branch Fst vs. Pixy GT Fst correlation") +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
branch_gt_cor_hist_by_winSize

ggsave(filename = snakemake@output[["branch_gt_cor_hist_by_winSize"]], plot = branch_gt_cor_hist_by_winSize,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

In [ ]:
site_gt_cor_hist_by_winSize <- all_fst_cors %>% 
    ggplot(aes(x = site_gt_cor)) +
        geom_histogram(bins = 30, color = "black", fill = "red") +
        ylab("Number of 1Mb regions") + xlab("ARG site Fst vs. Pixy GT Fst correlation") +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
site_gt_cor_hist_by_winSize

ggsave(filename = snakemake@output[["site_gt_cor_hist_by_winSize"]], plot = site_gt_cor_hist_by_winSize,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

In [ ]:
branch_sfs_cor_hist_by_winSize <- all_fst_cors %>% 
    ggplot(aes(x = branch_sfs_cor)) +
        geom_histogram(bins = 30, color = "black", fill = "red") +
        ylab("Number of 1Mb regions") + xlab("ARG branch Fst vs. ANGSD SFS Fst correlation") +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
branch_sfs_cor_hist_by_winSize

ggsave(filename = snakemake@output[["branch_sfs_cor_hist_by_winSize"]], plot = branch_sfs_cor_hist_by_winSize,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)

In [ ]:
site_sfs_cor_hist_by_winSize <- all_fst_cors %>% 
    ggplot(aes(x = site_sfs_cor)) +
        geom_histogram(bins = 30, color = "black", fill = "red") +
        ylab("Number of 1Mb regions") + xlab("ARG branch Fst vs. ANGSD SFS Fst correlation") +
        facet_wrap(~win_size, scales = "free", labeller = as_labeller(labels)) +
        theme_classic() +
        theme(axis.text = element_text(size = 13),
              axis.title = element_text(size = 15),
              strip.text.x = element_text(size = 13))
site_sfs_cor_hist_by_winSize

ggsave(filename = snakemake@output[["site_sfs_cor_hist_by_winSize"]], plot = site_sfs_cor_hist_by_winSize,
       device = "pdf", width = 14, height = 6, units = "in", dpi = 300)